In [1]:
import math
import json
import numpy as np
import falconn
import timeit
from __future__ import print_function
import psycopg2
import pandas as pd
from configparser import ConfigParser
import requests
import re
import traceback

In [8]:
def config(filename='prepare_data.ini', section='phonetic'):
    parser = ConfigParser()
    parser.read(filename)
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def db_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    params = config()
    conn = psycopg2.connect(**params)
    return conn

def read_dataframe():
    conn = db_connect()
    result = None
    try:
        query = """
            SELECT * FROM validation.validation_data
            where array_length(string_to_array(eng_variants, ','), 1) > 5
            or array_length(string_to_array(arb_yamli_variants, ','), 1) > 5
            """
        result = pd.read_sql(query, con=conn, index_col='id')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return result

def update_db_row(
    idx, 
    status, 
    login, 
    from_eng_eng_validated, 
    from_eng_eng_model_validated, 
    from_eng_arb_validated, 
    from_eng_arb_model_validated, 
    from_eng_arb_yamli_validated,
    from_arb_eng_validated, 
    from_arb_eng_model_validated, 
    from_arb_arb_validated, 
    from_arb_arb_model_validated, 
    from_arb_arb_yamli_validated):
    conn = db_connect()
    try:
        sql = """
            UPDATE validation.validation_data 
            SET status = %s, 
                login = %s, 
                from_eng_eng_validated = %s, 
                from_eng_eng_model_validated = %s, 
                from_eng_arb_validated = %s, 
                from_eng_arb_model_validated = %s, 
                from_eng_arb_yamli_validated = %s,
                from_arb_eng_validated = %s, 
                from_arb_eng_model_validated = %s, 
                from_arb_arb_validated = %s, 
                from_arb_arb_model_validated = %s, 
                from_arb_arb_yamli_validated = %s
            WHERE id = %s
            """
        cur = conn.cursor()
        cur.execute(sql, (status, 
                          login, 
                          from_eng_eng_validated, 
                          from_eng_eng_model_validated, 
                          from_eng_arb_validated, 
                          from_eng_arb_model_validated, 
                          from_eng_arb_yamli_validated, 
                          from_arb_eng_validated, 
                          from_arb_eng_model_validated, 
                          from_arb_arb_validated, 
                          from_arb_arb_model_validated, 
                          from_arb_arb_yamli_validated, 
                          idx))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
def read_db_row(idx):
    conn = db_connect()
    result = None
    try:
        sql = """
            SELECT 
                from_eng_eng_validated, 
                from_eng_eng_model_validated, 
                from_eng_arb_validated, 
                from_eng_arb_model_validated, 
                from_eng_arb_yamli_validated,
                from_arb_eng_validated, 
                from_arb_eng_model_validated, 
                from_arb_arb_validated, 
                from_arb_arb_model_validated, 
                from_arb_arb_yamli_validated
            FROM validation.validation_data 
            WHERE id = %s
            """
        cur = conn.cursor()
        cur.execute(sql, (idx, ))
        result = cur.fetchone()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return result
    
    
def claim_db_row(idx, login):
    conn = db_connect()
    try:
        sql = """
            UPDATE validation.validation_data 
            SET status = %s, login = %s 
            WHERE id = %s
            """
        cur = conn.cursor()
        cur.execute(sql, (int(1), login, int(idx)))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
def load_next_row(login):
    result = {}
    row = None
    
    x = df[df.status == 0]
    if len(x) > 0:
        row = x.iloc[0]
    if row is None:
        return None
    
    idx = int(row.name)
    
    df.set_value(col='status', index=idx, value=1)
    df.set_value(col='login', index=idx, value=login)
    row['status'] = int(1)
    row['login'] = login
    
    result['id'] = idx
    result['eng'] = row['eng']
    result['arb'] = row['arb']
    result['eng_variants'] = {name : 0 for name in row['eng_variants'].split(",")}
    result['eng_model_variants'] = {name : 0 for name in row['eng_model_variants'].split(",")}
    result['arb_variants'] = {name : 0 for name in row['arb_variants'].split(",")}
    result['arb_model_variants'] = {name : 0 for name in row['arb_model_variants'].split(",")}
    result['arb_yamli_variants'] = {name : 0 for name in row['arb_yamli_variants'].split(",")}
    
    claim_db_row(idx=idx, login=login)
    
    return result

def load_open_row_ids(login):
    x = df[(df['status'] == 1) & (df['login'] == login)]
    result = []
    for idx, row in x.iterrows():
        result.append(int(idx))
    return result

'''def load_row_by_id(idx):
    
    row = df.loc[[idx]]
    result = {}
       
    

    result['id'] = int(idx)
    result['eng'] = df.get_value(idx, 'eng')
    result['arb'] = df.get_value(idx, 'arb')
    result['eng_variants'] = {}
    result['eng_model_variants'] = {}
    result['arb_variants'] = {}
    result['arb_model_variants'] = {}
    result['arb_yamli_variants'] = {}

    s = df.get_value(idx, 'eng_variants')
    if s is not None and isinstance(s, str):
        for name in s.split(","):
            if (name.strip() != ""):
                result['eng_variants'][name] = 0

    s = df.get_value(idx, 'eng_model_variants')
    if s is not None and isinstance(s, str):
        for name in s.split(","):
            if (name.strip() != ""):
                result['eng_model_variants'][name] = 0

    s = df.get_value(idx, 'arb_variants')
    if s is not None and isinstance(s, str):
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_variants'][name] = 0

    s = df.get_value(idx, 'arb_model_variants')
    if s is not None and isinstance(s, str):
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_model_variants'][name] = 0

    s = df.get_value(idx, 'arb_yamli_variants')
    if s is not None and isinstance(s, str):
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_yamli_variants'][name] = 0
    
    validated = read_db_row(idx)
    
    #s = df.get_value(idx, 'eng_validated')
    s = validated[0]
    if s is not None:
        for name in s.split(","):
            if (name.strip() != ""):
                result['eng_variants'][name] = 1

    #s = df.get_value(idx, 'eng_model_validated')
    s = validated[1]
    if s is not None:
        for name in s.split(","):
            if (name.strip() != ""):
                result['eng_model_variants'][name] = 1

    #s = df.get_value(idx, 'arb_validated')   
    s = validated[2]
    if s is not None:
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_variants'][name] = 1
    
    #s = df.get_value(idx, 'arb_model_validated')
    s = validated[3]
    if s is not None:
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_model_variants'][name] = 1

    #s = df.get_value(idx, 'arb_yamli_validated')
    s = validated[4]
    if s is not None:
        for name in s.split(","):
            if (name.strip() != ""):
                result['arb_yamli_variants'][name] = 1 
    return result
'''
def update_row(idx, data, login, status):
    
    df.set_value(col='status', index=idx, value=status)
    df.set_value(col='login', index=idx, value=login)
    
    from_eng_arb_validated = data['from_eng_arb_validated']
    from_eng_arb_model_validated = data['from_eng_arb_model_validated']
    from_eng_arb_yamli_validated = data['from_eng_arb_yamli_validated']
    from_eng_eng_validated = data['from_eng_eng_validated']
    from_eng_eng_model_validated = data['from_eng_eng_model_validated']
    from_eng_arb = ''
    from_eng_arb_model = ''
    from_eng_arb_yamli = ''
    from_eng_eng = ''
    from_eng_eng_model = ''
    
    from_arb_arb_validated = data['from_arb_arb_validated']
    from_arb_arb_model_validated = data['from_arb_arb_model_validated']
    from_arb_arb_yamli_validated = data['from_arb_arb_yamli_validated']
    from_arb_eng_validated = data['from_arb_eng_validated']
    from_arb_eng_model_validated = data['from_arb_eng_model_validated']
    from_arb_arb = ''
    from_arb_arb_model = ''
    from_arb_arb_yamli = ''
    from_arb_eng = ''
    from_arb_eng_model = ''
    
    for x in from_eng_arb_validated:
        from_eng_arb = from_eng_arb + ',' + x
    from_eng_arb = from_eng_arb[1:]
    
    for x in from_eng_arb_yamli_validated:
        from_eng_arb_yamli = from_eng_arb_yamli + ',' + x
    from_eng_arb_yamli = from_eng_arb_yamli[1:]
    
    for x in from_eng_arb_model_validated:
        from_eng_arb_model = from_eng_arb_model + ',' + x
    from_eng_arb_model = from_eng_arb_model[1:]
    
    for x in from_eng_eng_validated:
        from_eng_eng = from_eng_eng + ',' + x
    from_eng_eng = from_eng_eng[1:]
    
    for x in from_eng_eng_model_validated:
        from_eng_eng_model = from_eng_eng_model + ',' + x
    from_eng_eng_model = from_eng_eng_model[1:]
    
    
    for x in from_arb_arb_validated:
        from_arb_arb = from_arb_arb + ',' + x
    from_arb_arb = from_arb_arb[1:]
    
    for x in from_arb_arb_yamli_validated:
        from_arb_arb_yamli = from_arb_arb_yamli + ',' + x
    from_arb_arb_yamli = from_arb_arb_yamli[1:]
    
    for x in from_arb_arb_model_validated:
        from_arb_arb_model = from_arb_arb_model + ',' + x
    from_arb_arb_model = from_arb_arb_model[1:]
    
    for x in from_arb_eng_validated:
        from_arb_eng = from_arb_eng + ',' + x
    from_arb_eng = from_arb_eng[1:]
    
    for x in from_arb_eng_model_validated:
        from_arb_eng_model = from_arb_eng_model + ',' + x
    from_arb_eng_model = from_arb_eng_model[1:]
    
    df.set_value(col='from_eng_arb_validated', index=idx, value=from_eng_arb)
    df.set_value(col='from_eng_arb_yamli_validated', index=idx, value=from_eng_arb_yamli)
    df.set_value(col='from_eng_arb_model_validated', index=idx, value=from_eng_arb_model)
    df.set_value(col='from_eng_eng_validated', index=idx, value=from_eng_eng)
    df.set_value(col='from_eng_eng_model_validated', index=idx, value=from_eng_eng_model)
    
    df.set_value(col='from_arb_arb_validated', index=idx, value=from_arb_arb)
    df.set_value(col='from_arb_arb_yamli_validated', index=idx, value=from_arb_arb_yamli)
    df.set_value(col='from_arb_arb_model_validated', index=idx, value=from_arb_arb_model)
    df.set_value(col='from_arb_eng_validated', index=idx, value=from_arb_eng)
    df.set_value(col='from_arb_eng_model_validated', index=idx, value=from_arb_eng_model)
    
    update_db_row(
        idx=int(idx), 
        login=login, 
        status=status, 
        from_eng_arb_validated=from_eng_arb, 
        from_eng_arb_model_validated=from_eng_arb_model, 
        from_eng_eng_validated=from_eng_eng, 
        from_eng_eng_model_validated=from_eng_eng_model,
        from_eng_arb_yamli_validated=from_eng_arb_yamli,
        from_arb_arb_validated=from_arb_arb, 
        from_arb_arb_model_validated=from_arb_arb_model, 
        from_arb_eng_validated=from_arb_eng, 
        from_arb_eng_model_validated=from_arb_eng_model,
        from_arb_arb_yamli_validated=from_arb_arb_yamli)

In [9]:
global df
df = read_dataframe()

In [ ]:
'''
row = load_next_row(login)

updated = dict()
updated['arb_invalidated'] = {}
updated['arb_yamli_invalidated'] = {}
updated['eng_invalidated'] = {}

i = 0
for a in row['arb_variants']:
    updated['arb_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['arb_yamli_variants']:
    updated['arb_yamli_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['eng_variants']:
    updated['eng_invalidated'][a] = i%2 
    i = i + 1

validate_row(row['id'], updated, 'cez')
'''

In [ ]:
# POST /validate

req = json.loads(REQUEST)
body = req['body']

login = 'unspecified'
index = -1
updated = None

if 'login' in body:
    login = body['login']
if 'index' in body:
    index = body['index']
if 'updated' in body:
    updated = body['updated']
    
result = dict()
msg = ''

if (updated is not None and index != -1):
    update_row(idx=index, data=updated, login=login, status=2)
    
result['data'] = load_next_row(login)
result['message'] = msg
    
print(json.dumps({'result': result}))

In [ ]:
# POST /next

req = json.loads(REQUEST)
body = req['body']

login = 'unspecified'
index = -1
updated = None

if 'login' in body:
    login = body['login']
if 'index' in body:
    index = body['index']
if 'updated' in body:
    updated = body['updated']

result = dict()
msg = ''

if (updated is not None and index != -1):
    update_row(idx=index, data=updated, login='', status=0)
    
result['data'] = load_next_row(login)
result['message'] = msg
    
print(json.dumps({'result': result}))

In [ ]:
# POST /prev

req = json.loads(REQUEST)
body = req['body']

login = ''
index = -1

result = dict()
msg = ' '

if 'login' in body:
    login = body['login']
if 'prev_rowid' in body:
    index = body['prev_rowid']
    
if index == -1:
    open_ids = load_open_row_ids(login)
    result['open_row_ids'] = open_ids
    result['data'] = load_row_by_id(open_ids[len(open_ids)-1])
else:
    result['data'] = load_row_by_id(index)
result['message'] = msg

print(json.dumps({'result': result}))

In [ ]:
REQUEST = """{\"path\": {},\"args\": {},\"body\":{\"id\":4,\"eng\":\"MARIAM\",\"arb\":\"مريم\",\"eng_variants\":{\"MARIEM\":0,\"MARYUM\":0,\"MAREAM\":0,\"MARIM\":0,\"MARIAME\":0,\"MERYEM\":0,\"MAREME\":0,\"MARRIAM\":0,\"MARYAM\":0,\"MAREM\":0,\"MURIMI\":0,\"MARIYAM\":0,\"MARIAM\":0,\"MERIEM\":0,\"MARIEME\":0,\"MARIUM\":0,\"MERIAM\":0,\"MARAIM\":0,\"MREEM\":0,\"MORIAM\":0,\"MREEEM\":0,\"MERYAM\":0,\"MARYEM\":0,\"MARYM\":0,\"MEREAM\":0,\"MIRIAM\":0},\"eng_model_variants\":{\"MARIEM\":0,\"MARYEM\":0,\"MARHEEMA\":0,\"MARAEM\":0,\"MAYRYAM\":0,\"MARIYM\":0,\"MARYIM\":0,\"MERHAM\":0,\"MARYAM.\":0,\"MEREAM\":0,\"MAREIEM\":0,\"MEREEM\":0,\"MIRHAMY\":0,\"MARYIMA\":0,\"MAREYM\":0,\"MARYAM\":0,\"MARIAM\":0,\"MARHAM\":0,\"MAREEMH\":0,\"MAREUM\":0,\"MARIAMU\":0,\"MERYEM\":0,\"MAREIM\":0,\"MARUEM\":0,\"MARHOM\":0,\"MARHUM\":0,\"MARYAMI\":0,\"MARAYM\":0,\"MIRYAM\":0,\"مريم\":0,\"MAREYEM\":0,\"MORHIMA\":0},\"arb_variants\":{\"مارام\":0,\"مارييم\":0,\"مرمه\":0,\"مرييام\":0,\"مريمه\":0,\"مرائم\":0,\"مرايم\":0,\"موريام\":0,\"مورم\":0,\"مارايم\":0,\"مارعم\":0,\"مارائم\":0,\"مرام\":0,\"ميريام\":0,\"مارم\":0,\"ماريم\":0,\"مارييام\":0,\"مرعيم\":0,\"مرمي\":0,\"مريم\":0,\"مارعيم\":0,\"مريام\":0,\"مرعم\":0,\"موريمي\":0,\"مريامه\":0,\"مراعم\":0,\"ماريمه\":0,\"ميريم\":0,\"ميرعم\":0,\"مرامي\":0,\"ماريمي\":0,\"ماراعم\":0,\"مورمي\":0,\"مارمي\":0,\"مريامي\":0,\"ماريام\":0,\"موريم\":0,\"مرييم\":0,\"مرامه\":0,\"مرم\":0,\"مريمي\":0,\"مريوم\":0,\"ماريوم\":0},\"arb_model_variants\":{\"مارييم\":0,\"موريم\":0,\"مرميا\":0,\"مريمك\":0,\"ميروم\":0,\"مارام\":0,\"مريمى\":0,\"ماريما\":0,\"مورام\":0,\"مورامى\":0,\"مارامى\":0,\"ميريام\":0,\"ماروم\":0,\"مريمو\":0,\"مرام\":0,\"مارهيم\":0,\"ماراما\":0,\"ميرامى\":0,\"ميرام\":0,\"مَريم\":0,\"ماريام\":0,\"مركم\":0,\"مريمه\":0,\"ماركم\":0,\"ماروما\":0,\"ماريمو\":0,\"ميراما\":0,\"ميريما\":0,\"مريم\":0,\"ميرييم\":0},\"arb_yamli_variants\":{\"مَرْيم\":0,\"مَرْيَم\":0,\"ماريام\":0,\"مَريم\":0,\"ميريام\":0,\"ماريم\":0,\"ماريَم\":0,\"مارم\":0,\"مارْيَم\":0,\"مريم\":0,\"مريام\":0,\"ماريوم\":0},\"login\":\"tester\",\"index\":\"4\",\"updated\":{\"from_eng_eng_invalidated\":[\"MARIEM\"],\"from_eng_eng_model_invalidated\":[\"MARIEM\"],\"from_eng_arb_invalidated\":[\"مارام\"],\"from_eng_arb_model_invalidated\":[\"مارييم\"],\"from_eng_arb_yamli_invalidated\":[\"مارييم\"],\"from_arb_eng_invalidated\":[\"MARIEM\"],\"from_arb_eng_model_invalidated\":[\"MARIEM\"],\"from_arb_arb_invalidated\":[\"مارام\"],\"from_arb_arb_model_invalidated\":[\"مارييم\"],\"from_arb_arb_yamli_invalidated\":[\"مارييم\"]}}}"""
REQUEST

In [ ]:
df

In [ ]:
load_next_row('cez')